In [ ]:
from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Boxplot, DateScale
from bqplot.traits import convert_to_date

from ipyleaflet import AwesomeIcon
def mouse_click(**kwargs):
    if kwargs.get('type') == 'click':
        location=kwargs.get('coordinates')
        
        point = geometry.Point(location[1], location[0])
        print(point)
        
        icon = AwesomeIcon(
                    name='fa-crosshairs',
                    marker_color='green',
                    icon_color='darkgreen',
                    spin=False
                )

        marker = Marker(location=location, icon=icon, rotation_angle=0, draggable=False)
        
        my_map.add_layer(marker) 

my_map.on_interaction(mouse_click)

def create_time_series_fig(title, dates, y, y1, min_val, max_val):
    fig_layout = Layout(width='100%', height='100%')
    fig_layout = Layout(width='auto', height='auto', max_height='120px', max_width='360px')

    tick_style = {'font-size': 7}

    s2_dates = [datetime.datetime.strptime(i, '%Y-%m-%d') for i in dates]
    date_x = convert_to_date(s2_dates)
    
    x_scale = DateScale(min = date_x[0], max = date_x[-1])
    y_scale = LinearScale(min = min_val, max = max_val)

    ndvi_line  = Lines(x=date_x, y=y, scales={"x": x_scale, "y": y_scale})
    sndvi_line = Lines(x=date_x, y=y1, scales={"x": x_scale, "y": y_scale})
    
    tick_values = np.linspace(min_val, max_val, 5)
    ax_y = Axis(label=name,   scale=y_scale, orientation="vertical", side="left", tick_values=tick_values, tick_style=tick_style)
    ax_x = Axis(label="Date", scale=x_scale, num_ticks=5, tick_style=tick_style)


    fig = Figure(layout=fig_layout, axes=[ax_x, ax_y], marks=[ndvi_line, sndvi_line], 
                           title=title, 
                           animation_duration=500, 
                           title_style = {'font-size': '8'},
                           fig_margin = dict(top=16, bottom=16, left=16, right=16)
                )
    return fig, ndvi_line, sndvi_line, ax_x, ax_y

def get_pixel_inspector_panel(dates):

    title = 'NDVI'
    y = np.zeros(len(dates))*np.nan
    fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y = create_time_series_fig(title, dates, y, y, 0, 1)
    title = 'LAI'
    y = np.zeros(len(dates))*np.nan
    fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y = create_time_series_fig(title, dates, y, y, 0, 3)
    figs = VBox([fig1, fig2])
    
    return fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y, fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y

In [ ]:
from ipywidgets import Play, jslink, SelectionSlider
import io
import pylab as plt
import matplotlib as mpl

daily_img = None
def on_change_date_slider(change):

    if (change['name'] == 'value') & (change['type'] == 'change'):
        value = change["new"]
        old = change['old']
        ind = change['owner'].index

        global daily_img, lai_dot

        # value = dates[ind]
        home = os.getcwd()
        cwd = '/files/' + '/'.join(home.split('/')[3:])
        base_url = my_map.window_url.split('/lab/')[0] + cwd + '/'

        # url = 'data/S2_thumbs/S2_%s_lai_%03d.png'%(field_id, value)
        url = ndvi_thumbs[ind]
        url = base_url + url
        
        x_min, y_min, x_max, y_max = bounds

        img_bounds = (y_min, x_max), (y_max, x_min)

        if daily_img is None:
            daily_img = ImageOverlay(
            url=url,
            bounds = img_bounds,
            # name = 'S2_%s_lai_png'%(field_id)
            )
            my_map.add_layer(daily_img)
            daily_img.url = url
            daily_img.bounds = img_bounds
        else:
            daily_img.url = url
            daily_img.bounds = img_bounds

def get_colorbar(vmin, vmax, label, cmap = plt.cm.RdYlGn):
    fig = plt.figure(figsize=(5, 2))
    ax = fig.add_axes([0.05, 0.8, 0.5, 0.07])
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = mpl.colorbar.ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
    lai_colorbar_f = io.BytesIO()
    plt.savefig(lai_colorbar_f, bbox_inches='tight', format='png', pad_inches=0)
    plt.close()
    image = lai_colorbar_f.getvalue()
    output = widgetIMG(value=image, format='png',)
    output.layout.object_fit = 'contain'
    # lai_label = Label('$$LAI [m^2/m^2]$$')
    colorbar_label = Label(label)
    colorbar_box = VBox([colorbar_label, output])
    colorbar_control = WidgetControl(widget=colorbar_box, position="bottomright")
    return colorbar_control


def get_play_slider(dates):
    
    play = Play(
        value=0,
        min=0,
        max=len(dates),
        step=1,
        interval=200,
        description="Press play",
        disabled=False
    )

    date_slider = SelectionSlider(options = dates, description='Date: ', style={'description_width': 'initial'}) 
    date_slider.observe(on_change_date_slider)
    jslink((play, 'value'), (date_slider, 'index'))
    play_label = Label('Click to play movie')
    play_box = HBox([play, date_slider])
    play_box = VBox([play_label, play_box])
    play_control = WidgetControl(widget=play_box, position="bottomright")

    return play_control



In [ ]:
from osgeo import gdal
from glob import glob
import matplotlib.cm as cm
from PIL import Image


def create_ndvi_thumbs(surs, aoi):
    cmap = cm.RdYlGn
    dest_folder = os.path.dirname(surs[0])
    ndvi_thumbs = []
    for filename in surs:
        data = gdal.Warp('', filename, format = 'MEM', 
                         resampleAlg = gdal.GRIORA_NearestNeighbour, 
                         xRes=10, yRes=10,
                         cropToCutline=False, cutlineDSName=aoi).ReadAsArray() * 1.

        cloud = gdal.Warp('', filename.replace('.tif', '_cloud.tif'), format = 'MEM', 
                          resampleAlg = gdal.GRIORA_NearestNeighbour, 
                          xRes=10, yRes=10,
                          cropToCutline=False, cutlineDSName=aoi).ReadAsArray()

        ndvi = (data[1] - data[0]) / (data[0] + data[1])
        valid_mask = (np.isfinite(ndvi)) & (cloud < 40)



        alpha = (valid_mask * 255.).astype(np.uint8)

        greyscale = cmap(ndvi / 1., bytes=True)
        greyscale[:, :, -1] = alpha

        img = Image.fromarray(greyscale, mode='RGBA')

        scale = 256 / img.height
        new_height = int(scale * img.height)
        new_width = int(scale * img.width)

        img = img.resize(( new_width, new_height), resample = Image.NEAREST)
        this_alpha = img.getchannel('A')
        img = img.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)
        mask = Image.eval(this_alpha, lambda a: 255 if a <= 5 else 0)

        # Paste the color of index 255 and use alpha as a mask
        img.paste(255, mask)
        img.info['transparency'] = 255

        date = filename.split('/')[-1].split('_')[1][:8]
        
        fname = dest_folder + '/S2_ndvi_%s.png'%(date)
        img.save(fname)
        ndvi_thumbs.append(fname)
    return ndvi_thumbs

In [1]:
from ipywidgets import SelectionRangeSlider, Layout, Accordion, Tab, Text, DatePicker, VBox, Button, Label
from ipyleaflet import Map, DrawControl, TileLayer, WidgetControl, ImageOverlay
from ipywidgets import Image as widgetIMG

from pyproj import Proj, Transformer

import json
import os
import ee
import shutil
import requests
import datetime
import numpy as np
from retry import retry
from shapely import geometry
from multiprocessing import Pool
from functools import partial

import sys
sys.path.insert(0, './python/')
from map_utils import debounce

ee.Initialize(opt_url = 'https://earthengine-highvolume.googleapis.com')



def get_s2_files(geom, start, end):
    criteria = ee.Filter.And( ee.Filter.geometry(geom), 
                              ee.Filter.date(start, end))

    s2_sur_images = ee.ImageCollection('COPERNICUS/S2_SR')\
                      .filter(criteria).aggregate_array('system:index').getInfo()

    s2_cloud_images = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")\
                        .filter(criteria).aggregate_array('system:index').getInfo()

    s2_ids = sorted(list(set(s2_sur_images) & set(s2_cloud_images)))
    return s2_ids

def get_s2_files(geom, start, end):
    criteria = ee.Filter.And( ee.Filter.geometry(geom), 
                              ee.Filter.date(start, end))

    s2_sur_images = ee.ImageCollection('COPERNICUS/S2_SR')\
                      .filter(criteria).getInfo()
    
    s2_sur_ids = []
    angs = []
    for i in s2_sur_images['features']:
        s2_id = i['properties']['system:index']
        sza = i['properties']['MEAN_SOLAR_ZENITH_ANGLE']
        saa = i['properties']['MEAN_SOLAR_AZIMUTH_ANGLE']
        vza = i['properties']['MEAN_INCIDENCE_ZENITH_ANGLE_B2']
        vaa = i['properties']['MEAN_INCIDENCE_AZIMUTH_ANGLE_B2']

        sza = np.cos(np.deg2rad(sza))
        vza = np.cos(np.deg2rad(vza))
        raa = ((vaa - saa) % 360) / 360

        s2_sur_ids.append(s2_id)
        angs.append([sza, vza, raa])
    s2_angs_dict = dict(zip(s2_sur_ids, angs))
    
    s2_cloud_ids = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")\
                            .filter(criteria).aggregate_array('system:index').getInfo()
    s2_ids = sorted(list(set(s2_sur_ids) & set(s2_cloud_ids)))
    
    return s2_ids, s2_angs_dict


@retry(tries=10, delay=1, backoff=2)
def download_image(inp):
    image_id, bands, filename, geom = inp
    # if not os.path.exists(filename):
    image = ee.Image(image_id)
    download_option = {'name': image_id.split('/')[-1], 
                       'scale': 10, 
                       'bands': bands,
                       'format': "GEO_TIFF",
                       'region': geom,
                       }
    url = image.getDownloadURL(download_option)
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", image_id)
    # else:
    #     print("Using previously downloaded: ", image_id)


def get_s2_over_field(field_name, geom, start, end, dest_dir = 'data/S2_sur_obs/'):
    s2_ids, s2_angs_dict = get_s2_files(geom, start, end)
    
    s2_images = [os.path.join('COPERNICUS/S2_SR', i) for i in s2_ids]
    s2_cloud_images = [os.path.join('COPERNICUS/S2_CLOUD_PROBABILITY', i) for i in s2_ids]

    s2_image_filenames = [os.path.join(dest_dir, 'S2_%s.tif'       % s2_id) for s2_id in s2_ids]
    s2_cloud_filenames = [os.path.join(dest_dir, 'S2_%s_cloud.tif' % s2_id) for s2_id in s2_ids]

    s2_bands = [['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12'], ] * len(s2_images)
    s2_bands = [['B4','B8'], ] * len(s2_images)
    s2_bands = [['B2','B3','B4','B5','B6','B7','B8', 'B8A','B11','B12']]* len(s2_images)
    cloud_bands = [['probability'],] * len(s2_cloud_images)

    images_to_downaload = s2_images + s2_cloud_images
    bands_to_download = s2_bands + cloud_bands
    filenames = s2_image_filenames + s2_cloud_filenames
    geoms = [geom, ] * len(filenames)
    inps = np.array([images_to_downaload, bands_to_download, filenames, geoms]).T
    not_need_to_do = np.array([os.path.exists(i) for i in filenames])
    inps = inps[~not_need_to_do]
    inps = inps.tolist()
    
    if len(inps) > 0: 
        pool = Pool(min(10, len(inps)))
        pool.map(download_image, inps)
        pool.close()
        pool.join()

    return filenames, s2_angs_dict



center = [9, 0]
zoom = 15
my_map = Map(center=center, zoom=zoom, max_zoom=17)

draw_control = DrawControl()
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "weight": 1,
        "fillOpacity": 0.2
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
draw_control.circlemarker = {}
draw_control.polyline = {}
# draw_control.circle = {
#     "shapeOptions": {
#         "fillColor": "#efed69",
#         "color": "#efed69",
#         "fillOpacity": 1.0
#     }
# }
# draw_control.rectangle = {
#     "shapeOptions": {
#         "fillColor": "#fca45d",
#         "color": "#fca45d",
#         "fillOpacity": 1.0
#     }
# }

defaultLayout=Layout(width='100%', height='640px')
my_map = Map(center=(9.3771, -0.6062), zoom=zoom, scroll_wheel_zoom=True, max_zoom = 18, layout=defaultLayout)
dark_matter_layer = TileLayer(url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}')
my_map.add_layer(dark_matter_layer)
my_map.add_control(draw_control)



# 

# dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
# options = [(i.strftime('%b'), i) for i in dates]
# time_slider = SelectionRangeSlider(
#     options=options,
#     index=(0, 11),
#     description='2018',
#     disabled=False
# )

# panel_box = VBox([fig_box, k_slider1, k_slider2, k_slider3], layout = box_layout)

# k_slider = FloatSlider(min=0, max=6, value=2,        # Opacity is valid in [0,1] range
#                orientation='horizontal',       # Vertical slider is what we want
#                readout=True,                # No need to show exact value
#                layout=Layout(width='80%'),
#                description='K: ', 
#                style={'description_width': 'initial'}) 

# panel_box = VBox([fig_box, k_slider], layout = box_layout)




# accordion = Accordion(children=[time_slider, time_slider], titles=('Slider', 'Text'))
# widget_control1 = WidgetControl(widget=accordion, position='bottomleft')
# my_map.add_control(widget_control1)

date_picker1 = DatePicker(
    description='Start Date',
    disabled=False
)
date_picker2 = DatePicker(
    description='End Date',
    disabled=False
)

download_button = Button(
    description='Download',
    disabled=True,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

box_layout = Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='100%')

info_layout = Layout(display='flex',
                flex_flow='row',
                align_items='flex-start',
                width='100%')

Download_info_label = Button(
    description='1. Draw a field on the map\n2.Pick the date range\n3.Click Donwload',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

Download_info_label = VBox([Label('1. Give a neme to the field'), 
                            Label('2. Draw a field on the map'), 
                            Label('3. Pick the date range'), 
                            Label('4. Click Donwload')])

field_name = Text(description='Field name: ')

download_box = VBox([VBox([Download_info_label], layout = info_layout), field_name, date_picker1, date_picker2, download_button], layout = box_layout)

accordion = Accordion(children=[download_box], selected_index=None)
accordion.set_title(0, 'Download S2 images')

widget_control1 = WidgetControl(widget=accordion, position='bottomleft')
my_map.add_control(widget_control1)


from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Boxplot, DateScale, Scatter
from bqplot.traits import convert_to_date

from ipyleaflet import AwesomeIcon
from ipywidgets import Play, jslink, SelectionSlider, HBox, VBox
import io
import pylab as plt
import matplotlib as mpl
from osgeo import gdal
from glob import glob
import matplotlib.cm as cm
from PIL import Image


def create_ndvi_thumbs(surs, aoi):
    ndvi_cmap = cm.RdYlGn
    dest_folder = os.path.dirname(surs[0])
    ndvi_thumbs = []
    for filename in surs:
        data = gdal.Warp('', filename, format = 'MEM', 
                         resampleAlg = gdal.GRIORA_NearestNeighbour, 
                         xRes=10, yRes=10,
                         cropToCutline=False, cutlineDSName=aoi).ReadAsArray() * 1.

        cloud = gdal.Warp('', filename.replace('.tif', '_cloud.tif'), format = 'MEM', 
                          resampleAlg = gdal.GRIORA_NearestNeighbour, 
                          xRes=10, yRes=10,
                          cropToCutline=False, cutlineDSName=aoi).ReadAsArray()

        ndvi = (data[6] - data[2]) / (data[6] + data[2])
        valid_mask = (np.isfinite(ndvi)) & (cloud < 40)



        alpha = (valid_mask * 255.).astype(np.uint8)

        greyscale = ndvi_cmap(ndvi / 1., bytes=True)
        greyscale[:, :, -1] = alpha

        img = Image.fromarray(greyscale, mode='RGBA')

        scale = 256 / img.height
        new_height = int(scale * img.height)
        new_width = int(scale * img.width)

        img = img.resize(( new_width, new_height), resample = Image.NEAREST)
        this_alpha = img.getchannel('A')
        img = img.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)
        mask = Image.eval(this_alpha, lambda a: 255 if a <= 5 else 0)

        # Paste the color of index 255 and use alpha as a mask
        img.paste(255, mask)
        img.info['transparency'] = 255

        date = filename.split('/')[-1].split('_')[1][:8]
        
        fname = dest_folder + '/S2_ndvi_%s.png'%(date)
        img.save(fname)
        ndvi_thumbs.append(fname)
    return ndvi_thumbs

daily_img = None
def on_change_date_slider(change):

    if (change['name'] == 'value') & (change['type'] == 'change'):
        value = change["new"]
        old = change['old']
        ind = change['owner'].index

        global daily_img, lai_dot

        # value = dates[ind]
        home = os.getcwd()
        cwd = '/files/' + '/'.join(home.split('/')[3:])
        base_url = my_map.window_url.split('/lab/')[0] + cwd + '/'

        # url = 'data/S2_thumbs/S2_%s_lai_%03d.png'%(field_id, value)
        url = ndvi_thumbs[ind]
        url = base_url + url
        
        x_min, y_min, x_max, y_max = bounds

        img_bounds = (y_min, x_max), (y_max, x_min)

        if daily_img is None:
            daily_img = ImageOverlay(
            url=url,
            bounds = img_bounds,
            # name = 'S2_%s_lai_png'%(field_id)
            )
            my_map.add_layer(daily_img)
            daily_img.url = url
            daily_img.bounds = img_bounds
        else:
            daily_img.url = url
            daily_img.bounds = img_bounds

def get_colorbar(vmin, vmax, label, colorbar_control = None):
    fig = plt.figure(figsize=(5, 2))
    ax = fig.add_axes([0.05, 0.8, 0.5, 0.07])
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = mpl.colorbar.ColorbarBase(ax, cmap=cm.RdYlGn, norm=norm, orientation='horizontal')
    lai_colorbar_f = io.BytesIO()
    plt.savefig(lai_colorbar_f, bbox_inches='tight', format='png', pad_inches=0)
    plt.close()
    image = lai_colorbar_f.getvalue()
    output = widgetIMG(value=image, format='png',)
    output.layout.object_fit = 'contain'
    # lai_label = Label('$$LAI [m^2/m^2]$$')
    colorbar_label = Label(label)
    colorbar_box = VBox([colorbar_label, output])
    if colorbar_control is None:
        colorbar_control = WidgetControl(widget=colorbar_box, position="bottomright")
    else:
        colorbar_control.widget = colorbar_box
    return colorbar_control


def get_play_slider(dates, play_control = None):
    
    play = Play(
        value=0,
        min=0,
        max=len(dates),
        step=1,
        interval=200,
        description="Press play",
        disabled=False
    )

    date_slider = SelectionSlider(options = dates, description='Date: ', style={'description_width': 'initial'}) 
    date_slider.observe(on_change_date_slider)
    jslink((play, 'value'), (date_slider, 'index'))
    play_label = Label('Click to play movie')
    play_box = HBox([play, date_slider])
    play_box = VBox([play_label, play_box])
    if play_control is None:
        play_control = WidgetControl(widget=play_box, position="bottomright")
    else:
        play_control.widget = play_box
    return play_control


def create_time_series_fig(title, dates, y, y1, min_val, max_val):
    fig_layout = Layout(width='100%', height='100%')
    fig_layout = Layout(width='auto', height='auto', max_height='120px', max_width='360px')

    tick_style = {'font-size': 7}

    s2_dates = [datetime.datetime.strptime(i, '%Y-%m-%d') for i in dates]
    date_x = convert_to_date(s2_dates)
    
    x_scale = DateScale(min = date_x[0], max = date_x[-1])
    y_scale = LinearScale(min = min_val, max = max_val)

    ndvi_line  = Lines(x=date_x, y=y, scales={"x": x_scale, "y": y_scale}, line_style='dotted', marker='circle', marker_size=4, colors = ['green'])
    sndvi_line = Lines(x=date_x, y=y1, scales={"x": x_scale, "y": y_scale})
    
    ndvi_line  = Scatter(x=date_x, y=y, scales={"x": x_scale, "y": y_scale}, default_size=4, colors = ['green'])#, line_style='dotted', marker='circle', marker_size=4, colors = ['green'])
    # sndvi_line = Scatter(x=date_x, y=y1, scales={"x": x_scale, "y": y_scale})
    
    
    tick_values = np.linspace(min_val, max_val, 5)
    ax_y = Axis(label=title,   scale=y_scale, orientation="vertical", side="left", tick_values=tick_values, tick_style=tick_style)
    ax_x = Axis(label="Date", scale=x_scale, num_ticks=5, tick_style=tick_style)


    fig = Figure(layout=fig_layout, axes=[ax_x, ax_y], marks=[ndvi_line, sndvi_line], 
                           title=title, 
                           animation_duration=500, 
                           title_style = {'font-size': '8'},
                           fig_margin = dict(top=16, bottom=16, left=16, right=16)
                )
    return fig, ndvi_line, sndvi_line, ax_x, ax_y

def get_pixel_inspector_panel(dates):

    title = 'NDVI'
    y = np.zeros(len(dates))*np.nan
    fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y = create_time_series_fig(title, dates, y, y, 0, 1)
    title = 'LAI'
    y = np.zeros(len(dates))*np.nan
    fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y = create_time_series_fig(title, dates, y, y, 0, 3)
    figs = VBox([fig1, fig2])
    
    return fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y, fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y

filenames = []
bounds = None
ndvi_thumbs = []
dates = None


def download_s2_image():
    global filenames, bounds, ndvi_thumbs, dates, ndvi_line, lai_line, s2_angs_dict
    filed_id = str(field_name.value)
    start_date = date_picker1.value.strftime('%Y-%m-%d')
    end_date = date_picker2.value.strftime('%Y-%m-%d')
    coords = draw_control.last_draw['geometry']['coordinates']
    poly = geometry.Polygon(coords[0])
    bounds = poly.bounds
    bounds_str = '%.05f_%.05f_%.05f_%.05f'%(poly.bounds)
    dest_dir = 'data/S2_sur_obs_%s/'%filed_id # + bounds_str
    # dest_dir = 'data/S2_sur_obs/'
    if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)

    with open(dest_dir + '/aoi.geosjon', 'w') as f:
        json.dump(draw_control.last_draw, f)  
        
    geom = ee.Geometry.Polygon(coords)
    filenames, s2_angs_dict = get_s2_over_field(filed_id, geom, start_date, end_date, dest_dir = dest_dir)

play_control = None 
colorbar_control = None 
fig_control = None

def create_thumbs_and_pixel_inspector(filenames, play_control = None, colorbar_control = None, fig_control = None):
    global ndvi_thumbs 
    global lai_line, ndvi_line
    surs = [i for i in filenames if '_cloud' not in i]
    aoi = os.path.dirname(surs[0]) + '/aoi.geosjon'
    ndvi_thumbs = create_ndvi_thumbs(surs, aoi)
    
    dates = [datetime.datetime.strptime(i.split('_')[-1][:8], '%Y%m%d').strftime('%Y-%m-%d') for i in ndvi_thumbs]
    
    play_control = get_play_slider(dates, play_control)
    colorbar_control = get_colorbar(0, 1, 'NDVI', colorbar_control)
    
    # full_dates = [datetime.datetime.strptime(i.split('/')[-1].split('_')[1][:15], '%Y%m%dT%H%M%S') for i in filenames]
    
    # fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y, fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y
    ret = get_pixel_inspector_panel(dates)
    fig1, ndvi_line, sndvi_line, fig1_ax_x, fig1_ax_y, fig2, lai_line, slai_line, fig2_ax_x, fig2_ax_y = ret
    fig_panel = VBox([fig1, fig2])
    if fig_control is None:
        fig_control = WidgetControl(widget=fig_panel, position="topright")
    else:
        fig_control.widget = fig_panel
    return play_control, colorbar_control, fig_control

# def observe_download_ready(change):
    
#     if (change['type'] == 'change') & (change['new'] == 'Finished!'):

@debounce(0.2)
def download(button):
    global play_control, colorbar_control, fig_control
    if draw_control.last_draw['geometry'] is None:
        print('Draw polygon for your field!')
    elif field_name.value == '':
        print('Give a name to your field!')
    elif date_picker1.value is None:
        print('Pick a start date!')
    elif date_picker2.value is None:
        print('Pick a end date!')
    else:
        print('start downloading')
        # download_button.disabled = False
        download_button.disabled = True
        download_button.button_style = 'info'
        download_button.description = 'Downloading...'
        download_s2_image()
        download_button.description = 'Finished!'
        download_button.button_style = 'success'
        
        if (play_control is None) | (colorbar_control is None) | (fig_control is None):
            play_control, colorbar_control, fig_control = create_thumbs_and_pixel_inspector(filenames)
            my_map.add_control(play_control)
            my_map.add_control(colorbar_control)
            my_map.add_control(fig_control)
        else:
            create_thumbs_and_pixel_inspector(filenames, play_control, colorbar_control, fig_control)
            
        
def refresh_download_button(change):
    
    if draw_control.last_draw['geometry'] is None:
        print('Draw polygon for your field!')
    elif field_name.value == '':
        print('Give a name to your field!')
    elif date_picker1.value is None:
        print('Pick a start date!')
    elif date_picker2.value is None:
        print('Pick a end date!')
    elif change['type'] == 'change':
        download_button.disabled = False
        download_button.button_style = 'primary'
        download_button.description = 'Download'


import numpy as np
f = np.load('data/nnLai.npz', allow_pickle=True)
arrModel = f.f.model

def affine_forward(x, w, b):
    """
    Forward pass of an affine layer
    :param x: input of dimension (D, )
    :param w: weights matrix of dimension (D, M)
    :param b: biais vector of dimension (M, )
    :return output of dimension (M, ), and cache needed for backprop
    """
    out = np.dot(x, w) + b
    cache = (x, w)
    return out, cache

def relu_forward(x):
    """ Forward ReLU
    """
    out = np.maximum(np.zeros(x.shape).astype(np.float32), x)
    cache = x
    return out, cache

def predict(inputs, arrModel):
    nLayers = int(len(arrModel) / 2)
    r = inputs
    for i in range(nLayers):
        w, b = arrModel[i*2], arrModel[i*2 + 1]
        a, _ = affine_forward(r, w, b) 
        r, _ = relu_forward(a)
    return r

from ipyleaflet import Marker
marker = None
def mouse_click(**kwargs):
    global marker
    if kwargs.get('type') == 'click':
        location=kwargs.get('coordinates')
        point = geometry.Point(location[1], location[0])
        print(point)
        
        if marker is None:
            marker = Marker(location=location, rotation_angle=0, draggable=False)
            my_map.add_layer(marker) 
        else:
            marker.location = location
        if len(filenames) > 0: 
            lat, lon = marker.location
            g = gdal.Open(filenames[0])
            geo_trans = g.GetGeoTransform()
            projectionRef = g.GetProjectionRef()

            pj1 = Proj(projectionRef)
            transformer = Transformer.from_crs('EPSG:4326', pj1.crs)
            x, y = transformer.transform(lat, lon)

            pix_x = int((x - geo_trans[0]) / geo_trans[1])
            pix_y = int((y - geo_trans[3]) / geo_trans[5])
            surs = [i for i in filenames if '_cloud' not in i]
            ndvis = []
            inputs = []
            for filename in surs:
                sza, vza, raa = s2_angs_dict[filename.split('/')[-1][3:-4]]
                print(filename)
                # try:
                g = gdal.Open(filename)
                pix_val = g.ReadAsArray(pix_x, pix_y, 1, 1).squeeze()
                g = gdal.Open(filename.replace('.tif', '_cloud.tif'))
                cloud_val = g.ReadAsArray(pix_x, pix_y, 1, 1).squeeze()
                if cloud_val < 60:
                    ndvi = (pix_val[6] - pix_val[2]) / (pix_val[6] + pix_val[2])
                    use_bands = [0, 1, 2, 3, 4, 5, 7, 8, 9]
                    # inputs = np.array([pix_val/10000, [np.cos(np.deg2rad(30)), np.cos(np.deg2rad(5)), 120/360.]])
                    inp = np.concatenate([pix_val/10000, [sza, vza, raa]])
                    inputs.append(inp)
                    
                else:
                    inputs.append(np.zeros(13) * np.nan)
                    ndvi = np.nan
                # except:
                #     ndvi = np.nan
                ndvis.append(ndvi)
                # print(ndvi)
            ndvis = np.array(ndvis)
            lai = predict(np.array(inputs), arrModel).ravel()
            # lai = 3.89 * ndvis - 0.11
            ndvi_line.y = ndvis
            lai_line.y = np.log(lai) * -2
    
my_map.on_interaction(mouse_click)

draw_control.observe(refresh_download_button)
field_name.observe(refresh_download_button)
date_picker1.observe(refresh_download_button)
date_picker2.observe(refresh_download_button)

download_button.on_click(download)
# download_button.observe(observe_download_ready)


my_map

Map(center=[9.3771, -0.6062], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [5]:
import sys
sys.path.insert(0, 'python')
from wofost_utils import get_era5_gee

year = 2021
lat = 9
lon = 0
get_era5_gee(year, lat, lon, dest_folder='data/')

python/wofost_utils.py:196: RuntimeWarning: Mean of empty slice
  humidity            = np.maximum(np.nanmean(para_vals[0], axis=1), 0)
python/wofost_utils.py:197: RuntimeWarning: All-NaN slice encountered
  temperature_2m_max  = np.nanmax (para_vals[1], axis=1)
python/wofost_utils.py:198: RuntimeWarning: All-NaN slice encountered
  temperature_2m_min  = np.nanmin (para_vals[1], axis=1)
python/wofost_utils.py:201: RuntimeWarning: Mean of empty slice
  wind                = np.nanmean(np.sqrt(para_vals[4]**2 + para_vals[5]**2), axis=1)


PosixPath('data/ERA5_Somewhere_9.00_0.00_2021.csv')

In [6]:
import pandas as pd
df = pd.read_csv('data/ERA5_Somewhere_9.00_0.00_2021.csv', skiprows=7)